In [1]:
import numpy as np

def HydrologicCondition(BT_MonthlyFlow, BT_AnnFlow, COND):
    
    # Bring in Hydrologic Condition Limits Data
    # fopen('./Input/HydroCondition_Limits.txt')
    HCL = np.loadtxt('./Input/HydroCondition_Limits.txt', delimiter = '\t')
    
    HydroLength = len(BT_MonthlyFlow)

    # fill in migration where applicable.
    # C 1   | C 2  | C 3 | C 4 | C 5  | C 6  | C 7   | C 8 | C9
    # Month |Year  | Day |Bf   | Goal | Dur  | Avail | Nat | Dur
    
    # Now fill the HydroCond matrix with date information
    HydroCond_Mon = np.ones((HydroLength, 6))
    HydroCond_Mon[:,:4] = BT_MonthlyFlow[:,:4]
    # Set the HydroCond for the first 9 months to NaN

    # Calculate the monthly hydrologic condition
    for j in range(HydroLength): # Need to start in the second month of the data set

        # This is an indexing variable
        month = int(HydroCond_Mon[j, 1])

        # Run through the other percentiles to see if the month fits a higher percentile
        for kk in range(5, 1, -1):
            
            if BT_MonthlyFlow[j, 5] >= HCL[month-1, kk-1]:
                
                HydroCond_Mon[j, 4] = kk
                
                break

    # Now define the hydrologic condition to implement the flow rules
    # This set of conditions is backward looking at the monthly step
    # So go ahead and shift things ahead one month
    # This will go in the HydroCondition column 6
    
    HydroCond_Mon[:, 5] = np.roll(HydroCond_Mon[:, 4], 1)

    # Clean-up some circshift issues from the last operation
    HydroCond_Mon[0, 5] = np.nan
    HydroCond_Mon[0, 5] = HydroCond_Mon[0, 4]

    # Now create the daily arrays from the monthly arrays for the Hydrologic Condition

    dateLength = int(np.sum(BT_MonthlyFlow[:,0]))
    # CCDaily = np.zeros((dateLength,5))
    SLRMOD7 = np.zeros((dateLength,1))
    SLRMOD7Flows = np.zeros((dateLength,1))
    SLRMOD8 = np.zeros((dateLength,1))

    if COND == 1:

        # Bring in climate change daily date range to build arrays
        CCDaily = np.loadtxt('./Input/Hist_Daily_DateArr_2.txt', delimiter = '\t')

    else:

        # Bring in climate change daily date range to build arrays
        CCDaily = np.loadtxt('./Input/CC_Daily_DateArr_3.txt', delimiter = '\t')

    # if FLOOD == 1: #if flood analysis

    #     # Bring in climate change daily date range to build arrays
    #     CCDaily = np.loadtxt('./Input/Dates_daily_projections.txt', delimiter = ',')
    
    HydroCond_Daily = np.copy(CCDaily)
    HydroCond_Daily_FlowRules = np.copy(CCDaily)
    
    DailyLength = len(HydroCond_Daily)
    
    for j in range(HydroLength):
        
        for jj in range(DailyLength):

            if HydroCond_Daily[jj, 0] == HydroCond_Mon[j, 1] and HydroCond_Daily[jj, 3] == HydroCond_Mon[j, 3]:

                HydroCond_Daily[jj, 4] = HydroCond_Mon[j, 4]
                HydroCond_Daily_FlowRules[jj, 4] = HydroCond_Mon[j, 5]

    # Now do the annual part
    # Bring in hydrologic conditions limits data
    # fopen('./Input/AnnHydroCondition_Limits.txt')
    AHCL = np.loadtxt('./Input/AnnHydroCondition_Limits.txt', delimiter = '\t')
    
    AnnHydroLength = len(BT_AnnFlow)
    
    HydroCond_Ann = np.ones((AnnHydroLength, 4))
    HydroCond_Ann[:, 0] = BT_AnnFlow[:, 0]
    HydroCond_Ann[:, 2] = BT_AnnFlow[:, 1]

    # Calculate the monthly hydrologic condition
    for j in range(AnnHydroLength): # Need to start in the second month of the data set

        # This is an indexing variable
        year = HydroCond_Ann[j, 1]

        # Run through the other percentiles to see if the year fits a higher percentile
        for kk in range(5, 1, -1):
            
            if BT_AnnFlow[j, 1] >= AHCL[kk-1]: #AHCL[year, kk-1]:
                
                HydroCond_Ann[j, 1] = kk
                
                break
    
    return HydroCond_Daily, HydroCond_Daily_FlowRules, HydroCond_Ann, HydroCond_Mon, SLRMOD7, SLRMOD7Flows, SLRMOD8